In [1]:
from pathlib import Path

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

executor = LocalCommandLineCodeExecutor(work_dir=work_dir)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="ALWAYS",
)

In [2]:
# Configure LLM using OllamaConfig
config_list = [{
    "model": "deepseek-r1:14b",
    "base_url": "http://localhost:11434",
    "api_type": "ollama"
}]

# Configure the assistant
llm_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0.1,
}

In [3]:
# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
For any script, always include the pip install for all imports to reduce the chance of errors. Also include the correct in each prompt response to ensure the user can execute the code correctly.
"""

import os

code_writer_agent = ConversableAgent(
    "code_writer",
    system_message=code_writer_system_message,
    llm_config=llm_config,
    code_execution_config=False,  # Turn off code execution for this agent.
    max_consecutive_auto_reply=5,
    human_input_mode="ALWAYS",
)

In [4]:
import pprint

chat_result = code_executor_agent.initiate_chat(
    code_writer_agent, message="""
    Scrape the contents from this URL: https://neetcode.io/practice?tab=neetcode150
    Fetch all the <tables> tags available in the page. Convert them to pandas dataframes.
    Concat all the dataframes into a single dataframe and write it to a CSV file.
    
    """
)

pprint.pprint(chat_result)

code_executor_agent (to code_writer):


    Scrape the contents from this URL: https://neetcode.io/practice?tab=neetcode150
    Fetch all the <tables> tags available in the page. Convert them to pandas dataframes.
    Concat all the dataframes into a single dataframe and write it to a CSV file.
    
    

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executor_agent):

<think>
Alright, I need to figure out how to help the user scrape the contents from the given URL using Python. The task is to fetch all the <table> tags, convert them into pandas DataFrames, concatenate them, and then save the result into a CSV file.

First, I'll consider the tools needed for web scraping. Beautiful Soup is a good choice for parsing HTML content. To fetch the webpage, requests library will be used. For handling data, pandas is essential since we're dealing with tables and DataFrames.

